In [92]:
import numpy as np
import pandas as pd
import re
import random
import os
import gzip
from collections import Counter, defaultdict, namedtuple

In [13]:
%cd /Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject
!pwd

/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject
/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject


In [49]:
cd data

/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject/nlp/data


In [50]:
os.getcwd()

'/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject/nlp/data'

In [51]:
pwd

'/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject/nlp/data'

In [52]:
ls

KB/
articles_with_names.json
articles_with_no_names_in_text.json
common_crawl_news_warc_gz_list.csv
example_inputs.tsv
example_inputs_long_names.tsv
example_inputs_long_names.tsv.gz
example_inputs_long_names_pos_and_neg.tsv.gz
example_inputs_pos_and_neg.tsv.gz
example_kb.tsv.gz
example_kb_all_supplier.tsv.gz
short_name_conversion.tsv
value_chain/


## Prep Corpus

In [68]:
corpus_in = pd.read_csv('example_inputs_pos_and_neg.tsv.gz', compression='gzip', header=0, sep='\t')
corpus_in.head()

,entity1,entity2,left,middle,right,article,label
0,Apple Inc,Samsung Electronics Co Ltd,\nThese reviews have been especially popular w...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
2,Reliance Industries Ltd,Bharti Airtel Ltd,"\nThe entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,Photo\nThe 5-4 decision brought to an end a l...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1


In [69]:
#Remove any \n from the rows 
corpus_in.replace("\n","",inplace=True)
corpus_in.replace('(\n)','',regex=True,inplace=True)
corpus_in.head()

,entity1,entity2,left,middle,right,article,label
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
2,Reliance Industries Ltd,Bharti Airtel Ltd,"The entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,PhotoThe 5-4 decision brought to an end a law...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1


In [71]:
#Create sentences
df_data_sentences = corpus_in[['left', 'entity1', "middle",'entity2','right']].astype(str).agg(' '.join, axis=1)
corpus_in['sentences'] = df_data_sentences.values
corpus_in.head()

,entity1,entity2,left,middle,right,article,label,sentences
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0,These reviews have been especially popular wit...
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...
2,Reliance Industries Ltd,Bharti Airtel Ltd,"The entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,"The entry of Reliance Jio, the telecoms arm of..."
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,The U.S. Supreme Court on Tuesday ruled that f...
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,PhotoThe 5-4 decision brought to an end a law...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,REUTERS/Aziz Ta Reliance Industries Ltd Photo...


In [72]:
corpus = corpus_in[['sentences','label']]
corpus.head()

,sentences,label
0,These reviews have been especially popular wit...,0
1,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,0
2,"The entry of Reliance Jio, the telecoms arm of...",0
3,The U.S. Supreme Court on Tuesday ruled that f...,1
4,REUTERS/Aziz Ta Reliance Industries Ltd Photo...,1


In [74]:
corpus.iloc[0,0]

'These reviews have been especially popular with companies like  Apple Inc  (AAPL.O) and  Samsung Electronics Co Ltd  (005930.KS) frequently hit with patent infringement suits.'

## Prep KB

In [80]:
kb_df = pd.read_csv('example_kb_all_supplier.tsv.gz', compression='gzip', 
                                 names= ['relation', 'subject', 'object'], sep='\t', quotechar='"')

In [81]:
kb_df.head()

,relation,subject,object
0,supplier of,Apple Inc,Seoul Semiconductor Co Ltd
1,supplier of,Koizumi furnitech Thailand Co Ltd,Seoul Semiconductor Co Ltd
2,supplier of,Amazon.com Inc,DS Smith PLC
3,supplier of,Procter & Gamble Co,DS Smith PLC
4,supplier of,Zymeworks Inc,DS Smith PLC


In [82]:
kb_df.groupby(['subject', 'object']).agg('size')

subject                          object                              
1-800-Flowers.Com Inc            Google LLC                              1
                                 International Business Machines Corp    1
1337 Software Oy                 Microsoft Corp                          1
180 Degree Capital Corp          NeoPhotonics Corp                       1
22 South Howard Loft Group, LLC  International Business Machines Corp    1
                                                                        ..
nCino LLC                        Salesforce.Com Inc                      1
nTelos Inc                       Lumos Networks Corp                     1
t'elenet'i ss                    Nokia Oyj                               1
                                 Telefonaktiebolaget LM Ericsson         1
thyssenkrupp AG                  Vodafone Group PLC                      1
Length: 18064, dtype: int64

In [86]:
# How many unique subjects, objects do we have?
len(kb_df), len(kb_df.subject.unique()), len(kb_df.object.unique())

(18086, 4811, 5310)

In [90]:
kb_df.subject.value_counts().head(), kb_df.object.value_counts().head()

(Microsoft Corp                524
 Apple Inc                     321
 Samsung Electronics Co Ltd    257
 Boeing Co                     246
 Walmart Inc                   243
 Name: subject, dtype: int64,
 Oracle Corp                       253
 Microsoft Corp                    252
 Google LLC                        191
 Boeing Co                         182
 Cisco Systems Inc (Pre-Merger)    174
 Name: object, dtype: int64)

In [95]:
## How many subjects also appear as objects in the supplier relation? 
counter = Counter()
for sbj in kb_df.subject.unique(): 
    df_temp = kb_df[kb_df.object == sbj]
    if len(df_temp) > 0:
        counter[sbj] = len(df_temp)

In [98]:
counter.most_common(20)

[('Oracle Corp', 253),
 ('Microsoft Corp', 252),
 ('Google LLC', 191),
 ('Boeing Co', 182),
 ('Cisco Systems Inc (Pre-Merger)', 174),
 ('Nokia Oyj', 163),
 ('International Business Machines Corp', 158),
 ('Telefonaktiebolaget LM Ericsson', 158),
 ('Intel Corp', 133),
 ('Airbus SE', 130),
 ('Pricewaterhousecoopers LLP', 127),
 ('Salesforce.Com Inc', 125),
 ('AT&T Inc', 120),
 ('Amazon.com Inc', 119),
 ('Huawei Technologies Co Ltd', 118),
 ('SAP SE', 109),
 ('BP PLC', 96),
 ('Samsung Electronics Co Ltd', 96),
 ('Glencore PLC', 85),
 ('Shell PLC', 83)]

In [103]:
len(counter), len(kb_df.subject.unique())

(1660, 4811)

First attempt: Split sets via following system:
1) Look at all subjects in KB, split these into sets as per the desired ratio
2) Split KB triples into the sets based on subject entity
3) Split corpus:
> If entity 1 in a split -> push corpus item into that split

In [174]:
#ISSUE
kb_df[kb_df.subject.astype(str).str.contains("Cisco")==True].subject.unique(), kb_df[kb_df.subject.astype(str).str.contains("Amazon")==True].subject.unique(), kb_df[kb_df.subject.astype(str).str.contains("Samsung")==True].subject.unique()

(array(['Cisco Systems Inc', 'Cisco Webex LLC',
        'Cisco Systems Inc (Pre-Merger)'], dtype=object),
 array(['Amazon.com Inc', 'Amazon Web Services Inc',
        'Amazon India Limited'], dtype=object),
 array(['Samsung Electronics Co Ltd', 'Samsung SDI Co Ltd',
        'Samsung Electro-Mechanics Co Ltd', 'Samsung Securities Co Ltd',
        'Samsung Telecommunications America LLC', 'Samsung Display Co Ltd',
        'Samsung Mobile Display Co Ltd', 'Samsung Electronics America Inc',
        'Samsung Semiconductor Inc', 'Samsung Heavy Industries Co Ltd'],
       dtype=object))

In [126]:
from sklearn.model_selection import train_test_split

In [146]:
np.random.choice(3, p = [0.1,0.3,0.6])

2

In [154]:
subjects = pd.DataFrame(kb_df.subject.unique()).rename(columns = {0: 'subject'})
subjects['split'] = [np.random.choice(3, p = [0.1,0.3,0.6]) for _ in range(len(subjects))]
subjects.head(10)

,subject,split
0,Apple Inc,2
1,Koizumi furnitech Thailand Co Ltd,1
2,Amazon.com Inc,1
3,Procter & Gamble Co,2
4,Zymeworks Inc,2
5,Nestle SA,2
6,Unilever PLC,2
7,SoftBank Group Corp,1
8,Sprint Corp,2
9,Total Access Communication PCL,1


In [155]:
subjects.split.value_counts()

2    2889
1    1451
0     471
Name: split, dtype: int64

In [158]:
subjects[subjects.subject.astype(str).str.contains('Cisco')]

,subject,split
234,Cisco Systems Inc,2
4669,Cisco Webex LLC,2
4810,Cisco Systems Inc (Pre-Merger),2


In [167]:
kb_with_splits = kb_df.merge(subjects, how = 'left', on = 'subject')
kb_with_splits

,relation,subject,object,split
0,supplier of,Apple Inc,Seoul Semiconductor Co Ltd,2
1,supplier of,Koizumi furnitech Thailand Co Ltd,Seoul Semiconductor Co Ltd,1
2,supplier of,Amazon.com Inc,DS Smith PLC,1
3,supplier of,Procter & Gamble Co,DS Smith PLC,2
4,supplier of,Zymeworks Inc,DS Smith PLC,2
...,...,...,...,...
18081,supplier of,Cisco Systems Inc (Pre-Merger),Synetrix (Holdings) Ltd,2
18082,supplier of,Cisco Systems Inc (Pre-Merger),Servcorp Ltd,2
18083,supplier of,Cisco Systems Inc (Pre-Merger),SolarWinds Corp,2
18084,supplier of,Cisco Systems Inc (Pre-Merger),Unified Communications Ltd,2


In [170]:
split_dict = {sbj: split for sbj, split in zip(kb_with_splits.subject, kb_with_splits.split)}
split_dict['Apple Inc']

2

In [176]:
corpus_with_splits = corpus_in.copy()

splits = []
count_not_matched = 0
for E1, E2 in zip(corpus_in.entity1, corpus_in.entity2): 
    if E1 in split_dict.keys():
        splits.append(split_dict[E1])
    elif E2 in split_dict.keys():
        splits.append(split_dict[E2])
    else:
        splits.append(np.random.choice(3, p = [0.1,0.3,0.6]))
        print('{}, {} not in KB split' .format(E1, E2))
        count_not_matched += 1
print('Total of {} not macthed' .format(count_not_matched))


Pivotal Software Inc, Carbon Black Inc not in KB split
Amphenol Corp, TE Connectivity Ltd not in KB split
Samsung Group, Samsung C&T Corp not in KB split
Samsung Group, Samsung C&T Corp not in KB split
Cheniere Energy Inc, Mobil Corp not in KB split
Inphi Corp, Finisar Corp not in KB split
Eastman Kodak Co, Ally Financial Inc not in KB split
Albemarle Corp, Livent Corp not in KB split
Motherson Sumi Systems Ltd, Leoni AG not in KB split
Leoni AG, Motherson Sumi Systems Ltd not in KB split
Motherson Sumi Systems Ltd, Leoni AG not in KB split
Leoni AG, Motherson Sumi Systems Ltd not in KB split
Motherson Sumi Systems Ltd, Leoni AG not in KB split
Leoni AG, Motherson Sumi Systems Ltd not in KB split
Motherson Sumi Systems Ltd, Leoni AG not in KB split
Martinrea International Inc, Linamar Corp not in KB split
Samsung C&T Corp, Samsung Group not in KB split
Samsung C&T Corp, Samsung Group not in KB split
Samsung Group, Samsung C&T Corp not in KB split
Samsung C&T Corp, Samsung Group not in 

In [178]:
corpus_with_splits['split'] = splits
corpus_with_splits.head(10)

,entity1,entity2,left,middle,right,article,label,sentences,split
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0,These reviews have been especially popular wit...,2
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,0
2,Reliance Industries Ltd,Bharti Airtel Ltd,"The entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,"The entry of Reliance Jio, the telecoms arm of...",2
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,The U.S. Supreme Court on Tuesday ruled that f...,0
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,PhotoThe 5-4 decision brought to an end a law...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,REUTERS/Aziz Ta Reliance Industries Ltd Photo...,2
5,Amazon.com Inc,Alphabet Inc,The in-home delivery option put,in direct competition with so-called smart ho...,'s (GOOGL.O) Nest Labs and presaged Amazon's $...,884c15dbfd38525ffd89c79ae9f247ac828382bf39e2e2...,1,The in-home delivery option put Amazon.com In...,1
6,Alphabet Inc,Amazon.com Inc,The in-home delivery option put Amazon in dire...,'s (GOOGL.O) Nest Labs and presaged,'s $1 billion acquisition of connected doorbel...,884c15dbfd38525ffd89c79ae9f247ac828382bf39e2e2...,1,The in-home delivery option put Amazon in dire...,2
7,Amazon.com Inc,General Motors Co,All that is required to have packages delivere...,and linking it to the vehicle's so-called con...,'s (GM.N) OnStar system or Volvo Car Group's O...,884c15dbfd38525ffd89c79ae9f247ac828382bf39e2e2...,1,All that is required to have packages delivere...,1
8,General Motors Co,Amazon.com Inc,All that is required to have packages delivere...,'s (GM.N) OnStar system or Volvo Car Group's O...,'s drive to leave packages where they cannot b...,884c15dbfd38525ffd89c79ae9f247ac828382bf39e2e2...,1,All that is required to have packages delivere...,2
9,Viavi Solutions Inc,Finisar Corp,According to parts manufacturers,",","and Ams AG, bottlenecks on key parts will mea...",b87f0cf775e9bf0419dd5298dd8339166518f0dd60717b...,0,According to parts manufacturers Viavi Soluti...,2
